## Importing NIH Dataset (ZIP Format)

In [1]:
# Import relevant packages
import numpy as np
import os
from shutil import copyfile
from zipfile import ZipFile

# Download NIH dataset zip file
!wget -nc ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip

# Extract images if not already extracted
ROOT_DIR = os.path.join("/", "content")
if not os.path.isdir("cell_images"):
    print("Extracting images...")
    with ZipFile(os.path.join("cell_images.zip"), "r") as zipObj:
        zipObj.extractall()
    print("Done!")

--2020-03-05 00:26:15--  ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip
           => ‘cell_images.zip’
Resolving lhcftp.nlm.nih.gov (lhcftp.nlm.nih.gov)... 130.14.55.35, 2607:f220:41e:7055::35
Connecting to lhcftp.nlm.nih.gov (lhcftp.nlm.nih.gov)|130.14.55.35|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /Open-Access-Datasets/Malaria ... done.
==> SIZE cell_images.zip ... 353452851
==> PASV ... done.    ==> RETR cell_images.zip ... done.
Length: 353452851 (337M) (unauthoritative)

cell_images.zip     100%[===================>] 337.08M  65.7MB/s    in 5.4s    

2020-03-05 00:26:22 (62.9 MB/s) - ‘cell_images.zip’ saved [353452851]

Extracting images...
Done!


## Unzip Images, Resize, and Store in NumPy Arrays

In [2]:
# Install and import relevant packages
import numpy as np
import os
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6 libxrender1
import cv2
from PIL import Image

# Create new folders to save rescaled images
if not os.path.isdir("RescaledSet"):
    os.mkdir("RescaledSet")
if not os.path.isdir("RescaledSet/Parasitized"):
    os.mkdir("RescaledSet/Parasitized")
if not os.path.isdir("RescaledSet/Uninfected"):
    os.mkdir("RescaledSet/Uninfected")

# Generate list of parasitized file names
ParasitizedFiles = os.listdir("cell_images/Parasitized/")
UninfectedFiles = os.listdir("cell_images/Uninfected/")

# Remove Thumb.db files
while 'Thumbs.db' in ParasitizedFiles: ParasitizedFiles.remove('Thumbs.db')   
while 'Thumbs.db' in UninfectedFiles: UninfectedFiles.remove('Thumbs.db')  

# Pre-allocate memory space for images
Parasitized = np.empty([13779,128,128,3])
Uninfected = np.empty([13779,128,128,3])

# Resize and load parasitized images
for i in range(13779):
    TempImage = cv2.imread('cell_images/Parasitized/'+ParasitizedFiles[i])
    ResizedImage = cv2.resize(TempImage, dsize=(128,128))
    Parasitized[i,:,:,:] = ResizedImage

# Resize and load uninfected images
for i in range(13779):
    TempImage = cv2.imread('cell_images/Uninfected/'+UninfectedFiles[i])
    ResizedImage = cv2.resize(TempImage, dsize=(128,128))
    Uninfected[i,:,:,:] = ResizedImage
    
print('Uninfected Dataset size is:',np.shape(Uninfected))
print('Parasitized Dataset size is:',np.shape(Parasitized))


# Generate image dataset
Dataset = np.concatenate((Parasitized, Uninfected), axis=0)
del Parasitized
del Uninfected

     |████████████████████████████████| 28.2 MB 3.4 MB/s eta 0:00:01
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease        
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done                 
Building dependency tree       
Reading state information... Done
28 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The 

## Generate Cross-Validation Indices for Training and Testing Sets

In [3]:
# Generate 5-fold cross-validation groups
CVIndices = np.random.permutation(Dataset.shape[0])
Index1, Index2, Index3, Index4, Index5 = CVIndices[:5512], CVIndices[5512:11024], CVIndices[11024:16536], CVIndices[16536:22048], CVIndices[22048:]

## Train Model and Save Results as CSV Files (4 Mapping Blocks)

In [4]:
High = [48,56]
Low = [12,16]

In [ ]:
# Import relevant packages for neural network training
import sys
import csv
if 'tensorflow' in sys.modules == False:
    %tensorflow_version 2.x
    import tensorflow as tf
import keras
## Create FSRCNN architecture
from keras import optimizers
from keras.models import load_model
from keras.models import Sequential, Model
from keras.layers import Dense, Activation
from keras.layers import Conv2D, MaxPooling2D, Input, ZeroPadding2D, Conv2DTranspose, merge 
from keras.layers.advanced_activations import PReLU
from keras.preprocessing import image

!pip install scikit-image
from skimage.transform import rescale, resize, downscale_local_mean


for d in High:
    for s in Low:

        # Create empty lists to store results
        TrainLoss = []
        TestLoss = []
        
        for p in range(5):

            # Create the appropriate training and testing sets
            if i == 0:
                TrainOut = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:]), axis=0)
                TestOut = Dataset[Index5,:]
            elif i == 1:
                TrainOut = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index5,:]), axis=0)
                TestOut = Dataset[Index4,:]
            elif i == 2:
                TrainOut = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
                TestOut = Dataset[Index3,:]
            elif i == 3:
                TrainOut = np.concatenate((Dataset[Index1,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
                TestOut = Dataset[Index2,:]
            else:
                TrainOut = np.concatenate((Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
                TestOut = Dataset[Index1,:]


            # Generate train and test sets
            TrainIn = np.zeros([np.shape(TrainOut)[0],32,32,3])
            TestIn = np.zeros([np.shape(TestOut)[0],32,32,3])
            for i in range(np.shape(TrainOut)[0]):
                TrainIn[i,:,:,:] = downscale_local_mean(TrainOut[i,:,:,:], (4,4,1))
            for i in range(np.shape(TestOut)[0]):
                TestIn[i,:,:,:] = downscale_local_mean(TestOut[i,:,:,:], (4,4,1))       


            #Feature Extraction
            model = Sequential()
            input_img = Input(shape=(32,32,3))
            model = Conv2D(filters = d, kernel_size = (5, 5), padding='same', kernel_initializer='he_normal')(input_img)
            model = PReLU()(model)

            #Shrink
            model = Conv2D(filters = 16, kernel_size = (1, 1), padding='same', kernel_initializer='he_normal')(model)
            model = PReLU()(model)

            #Mapping
            model = Conv2D(filters = s, kernel_size = (3, 3), padding='same', kernel_initializer='he_normal')(model)
            model = PReLU()(model)
            model = Conv2D(filters = s, kernel_size = (3, 3), padding='same', kernel_initializer='he_normal')(model)
            model = PReLU()(model)
            model = Conv2D(filters = s, kernel_size = (3, 3), padding='same', kernel_initializer='he_normal')(model)
            model = PReLU()(model)
            model = Conv2D(filters = s, kernel_size = (3, 3), padding='same', kernel_initializer='he_normal')(model)
            model = PReLU()(model)

            #Exapansion
            model = Conv2D(filters = d, kernel_size = (1, 1), padding='same', kernel_initializer='he_normal')(model)
            model = PReLU()(model)

            #Deconvolution
            model = Conv2DTranspose(filters = 3, kernel_size = (9, 9), strides=(4, 4), padding='same')(model)
            output_img = model

            model = Model(input_img, output_img) #Create the model object
            adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False) #Training optimizer
            model.compile(loss = "mean_squared_error", optimizer = adam, metrics=["mean_squared_error"]) #How we measure error

            #model.summary()

            # Train model and evaluate performance
            print('We are now training cross-validation set #',p+1)
            Results = model.fit(y=TrainOut, x=TrainIn, validation_data = (TestIn,TestOut), epochs=100, batch_size = 32, validation_freq=1)


            # Display and store performance results
            Results.history['loss'] = [round(l, 4) for l in Results.history['loss']]
            Results.history['val_loss'] = [round(l, 4) for l in Results.history['val_loss']]

            print('Training Loss:',Results.history['loss'])
            print('Validation Loss:',Results.history['val_loss'])

            TrainLoss.append(Results.history['loss'])
            TestLoss.append(Results.history['val_loss'])
            print('')


        # Save and export as CSV files
        with open(str(d)+"_"+str(s)+"_4Maps_TrainLoss.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(TrainLoss)
        with open(str(d)+"_"+str(s)+"_4Maps_TestLoss.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(TestLoss)

We are now training cross-validation set # 1
Train on 22046 samples, validate on 5512 samples
Epoch 1/100
22046/22046 [==============================] - 11s 502us/step - loss: 594.9301 - mean_squared_error: 594.9299 - val_loss: 203.7362 - val_mean_squared_error: 203.7362
Epoch 2/100
22046/22046 [==============================] - 11s 510us/step - loss: 166.1296 - mean_squared_error: 166.1295 - val_loss: 146.1756 - val_mean_squared_error: 146.1755
Epoch 3/100
22046/22046 [==============================] - 12s 526us/step - loss: 134.3659 - mean_squared_error: 134.3660 - val_loss: 128.6632 - val_mean_squared_error: 128.6633
Epoch 4/100
22046/22046 [==============================] - 12s 527us/step - loss: 119.3706 - mean_squared_error: 119.3706 - val_loss: 117.2593 - val_mean_squared_error: 117.2593
Epoch 5/100
22046/22046 [==============================] - 11s 507us/step - loss: 111.1438 - mean_squared_error: 111.1438 - val_loss: 106.5403 - val_mean_squared_error: 106.5403
Epoch 6/100
2204

In [ ]:
# Import relevant packages for neural network training
import sys
import csv
if 'tensorflow' in sys.modules == False:
    %tensorflow_version 2.x
    import tensorflow as tf
import keras
## Create FSRCNN architecture
from keras import optimizers
from keras.models import load_model
from keras.models import Sequential, Model
from keras.layers import Dense, Activation
from keras.layers import Conv2D, MaxPooling2D, Input, ZeroPadding2D, Conv2DTranspose, merge 
from keras.layers.advanced_activations import PReLU
from keras.preprocessing import image

!pip install scikit-image
from skimage.transform import rescale, resize, downscale_local_mean


for d in High:
    for s in Low:

        # Create empty lists to store results
        TrainLoss = []
        TestLoss = []
        
        for p in range(5):

            # Create the appropriate training and testing sets
            if i == 0:
                TrainOut = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:]), axis=0)
                TestOut = Dataset[Index5,:]
            elif i == 1:
                TrainOut = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index5,:]), axis=0)
                TestOut = Dataset[Index4,:]
            elif i == 2:
                TrainOut = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
                TestOut = Dataset[Index3,:]
            elif i == 3:
                TrainOut = np.concatenate((Dataset[Index1,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
                TestOut = Dataset[Index2,:]
            else:
                TrainOut = np.concatenate((Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
                TestOut = Dataset[Index1,:]


            # Generate train and test sets
            TrainIn = np.zeros([np.shape(TrainOut)[0],32,32,3])
            TestIn = np.zeros([np.shape(TestOut)[0],32,32,3])
            for i in range(np.shape(TrainOut)[0]):
                TrainIn[i,:,:,:] = downscale_local_mean(TrainOut[i,:,:,:], (4,4,1))
            for i in range(np.shape(TestOut)[0]):
                TestIn[i,:,:,:] = downscale_local_mean(TestOut[i,:,:,:], (4,4,1))       


            #Feature Extraction
            model = Sequential()
            input_img = Input(shape=(32,32,3))
            model = Conv2D(filters = d, kernel_size = (5, 5), padding='same', kernel_initializer='he_normal')(input_img)
            model = PReLU()(model)

            #Shrink
            model = Conv2D(filters = 16, kernel_size = (1, 1), padding='same', kernel_initializer='he_normal')(model)
            model = PReLU()(model)

            #Mapping
            model = Conv2D(filters = s, kernel_size = (3, 3), padding='same', kernel_initializer='he_normal')(model)
            model = PReLU()(model)
            model = Conv2D(filters = s, kernel_size = (3, 3), padding='same', kernel_initializer='he_normal')(model)
            model = PReLU()(model)
            model = Conv2D(filters = s, kernel_size = (3, 3), padding='same', kernel_initializer='he_normal')(model)
            model = PReLU()(model)

            #Exapansion
            model = Conv2D(filters = d, kernel_size = (1, 1), padding='same', kernel_initializer='he_normal')(model)
            model = PReLU()(model)

            #Deconvolution
            model = Conv2DTranspose(filters = 3, kernel_size = (9, 9), strides=(4, 4), padding='same')(model)
            output_img = model

            model = Model(input_img, output_img) #Create the model object
            adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False) #Training optimizer
            model.compile(loss = "mean_squared_error", optimizer = adam, metrics=["mean_squared_error"]) #How we measure error

            #model.summary()

            # Train model and evaluate performance
            print('We are now training cross-validation set #',p+1)
            Results = model.fit(y=TrainOut, x=TrainIn, validation_data = (TestIn,TestOut), epochs=100, batch_size = 32, validation_freq=1)


            # Display and store performance results
            Results.history['loss'] = [round(l, 4) for l in Results.history['loss']]
            Results.history['val_loss'] = [round(l, 4) for l in Results.history['val_loss']]

            print('Training Loss:',Results.history['loss'])
            print('Validation Loss:',Results.history['val_loss'])

            TrainLoss.append(Results.history['loss'])
            TestLoss.append(Results.history['val_loss'])
            print('')


        # Save and export as CSV files
        with open(str(d)+"_"+str(s)+"_3Maps_TrainLoss.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(TrainLoss)
        with open(str(d)+"_"+str(s)+"_3Maps_TestLoss.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(TestLoss)

In [ ]:
# Import relevant packages for neural network training
import sys
import csv
if 'tensorflow' in sys.modules == False:
    %tensorflow_version 2.x
    import tensorflow as tf
import keras
## Create FSRCNN architecture
from keras import optimizers
from keras.models import load_model
from keras.models import Sequential, Model
from keras.layers import Dense, Activation
from keras.layers import Conv2D, MaxPooling2D, Input, ZeroPadding2D, Conv2DTranspose, merge 
from keras.layers.advanced_activations import PReLU
from keras.preprocessing import image

!pip install scikit-image
from skimage.transform import rescale, resize, downscale_local_mean


for d in High:
    for s in Low:

        # Create empty lists to store results
        TrainLoss = []
        TestLoss = []
        
        for p in range(5):

            # Create the appropriate training and testing sets
            if i == 0:
                TrainOut = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:]), axis=0)
                TestOut = Dataset[Index5,:]
            elif i == 1:
                TrainOut = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index5,:]), axis=0)
                TestOut = Dataset[Index4,:]
            elif i == 2:
                TrainOut = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
                TestOut = Dataset[Index3,:]
            elif i == 3:
                TrainOut = np.concatenate((Dataset[Index1,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
                TestOut = Dataset[Index2,:]
            else:
                TrainOut = np.concatenate((Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
                TestOut = Dataset[Index1,:]


            # Generate train and test sets
            TrainIn = np.zeros([np.shape(TrainOut)[0],32,32,3])
            TestIn = np.zeros([np.shape(TestOut)[0],32,32,3])
            for i in range(np.shape(TrainOut)[0]):
                TrainIn[i,:,:,:] = downscale_local_mean(TrainOut[i,:,:,:], (4,4,1))
            for i in range(np.shape(TestOut)[0]):
                TestIn[i,:,:,:] = downscale_local_mean(TestOut[i,:,:,:], (4,4,1))       


            #Feature Extraction
            model = Sequential()
            input_img = Input(shape=(32,32,3))
            model = Conv2D(filters = d, kernel_size = (5, 5), padding='same', kernel_initializer='he_normal')(input_img)
            model = PReLU()(model)

            #Shrink
            model = Conv2D(filters = 16, kernel_size = (1, 1), padding='same', kernel_initializer='he_normal')(model)
            model = PReLU()(model)

            #Mapping
            model = Conv2D(filters = s, kernel_size = (3, 3), padding='same', kernel_initializer='he_normal')(model)
            model = PReLU()(model)
            model = Conv2D(filters = s, kernel_size = (3, 3), padding='same', kernel_initializer='he_normal')(model)
            model = PReLU()(model)

            #Exapansion
            model = Conv2D(filters = d, kernel_size = (1, 1), padding='same', kernel_initializer='he_normal')(model)
            model = PReLU()(model)

            #Deconvolution
            model = Conv2DTranspose(filters = 3, kernel_size = (9, 9), strides=(4, 4), padding='same')(model)
            output_img = model

            model = Model(input_img, output_img) #Create the model object
            adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False) #Training optimizer
            model.compile(loss = "mean_squared_error", optimizer = adam, metrics=["mean_squared_error"]) #How we measure error

            #model.summary()

            # Train model and evaluate performance
            print('We are now training cross-validation set #',p+1)
            Results = model.fit(y=TrainOut, x=TrainIn, validation_data = (TestIn,TestOut), epochs=100, batch_size = 32, validation_freq=1)


            # Display and store performance results
            Results.history['loss'] = [round(l, 4) for l in Results.history['loss']]
            Results.history['val_loss'] = [round(l, 4) for l in Results.history['val_loss']]

            print('Training Loss:',Results.history['loss'])
            print('Validation Loss:',Results.history['val_loss'])

            TrainLoss.append(Results.history['loss'])
            TestLoss.append(Results.history['val_loss'])
            print('')


        # Save and export as CSV files
        with open(str(d)+"_"+str(s)+"_2Maps_TrainLoss.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(TrainLoss)
        with open(str(d)+"_"+str(s)+"_2Maps_TestLoss.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(TestLoss)

## Examine RAM Requirements

In [ ]:
# Examine RAM Usage
import sys
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)